In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
import h5py
from tqdm import tqdm
import random

# -------------------------
# Squeeze-and-Excitation Block
# -------------------------
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.GELU(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return x * self.fc(x)

# -------------------------
# Attention Gate for Skip Connections
# -------------------------
class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super().__init__()
        self.W_g = nn.Conv2d(F_g, F_int, kernel_size=1, bias=False)
        self.W_x = nn.Conv2d(F_l, F_int, kernel_size=1, bias=False)
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, bias=True),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)
    def forward(self, g, x):
        # g: gating signal, x: skip connection
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

# -------------------------
# Depthwise Separable Conv (for ASPP)
# -------------------------
class SepConv(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, padding, dilation=1):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size, padding=padding,
                                   dilation=dilation, groups=in_ch, bias=False)
        self.pointwise = nn.Conv2d(in_ch, out_ch, 1, bias=False)
        self.norm = nn.GroupNorm(8, out_ch)
        self.act = nn.ReLU()
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return self.act(self.norm(x))

# -------------------------
# Multi-Scale Fusion
# -------------------------
class MultiScaleFusion(nn.Module):
    def __init__(self, in_ch, out_ch, scales=(1.0,0.5,0.25)):
        super().__init__()
        self.scales = scales
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False),
                nn.GroupNorm(8, out_ch), nn.ReLU(),
                SEBlock(out_ch)
            ) for _ in scales
        ])
        self.merge = nn.Sequential(
            nn.Conv2d(len(scales)*out_ch, out_ch, 1, bias=False),
            nn.GroupNorm(8, out_ch), nn.ReLU(),
            SEBlock(out_ch)
        )
    def forward(self, x):
        bs,c,h,w = x.shape
        outs = []
        for s,conv in zip(self.scales,self.convs):
            xi = F.interpolate(x, scale_factor=s, mode='bilinear', align_corners=False) if s!=1.0 else x
            if s!=1.0:
                xi = F.interpolate(xi, size=(h,w), mode='bilinear', align_corners=False)
            outs.append(conv(xi))
        return self.merge(torch.cat(outs, dim=1))

# -------------------------
# ASPP Module with Mixed Kernels
# -------------------------
class ASPP(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        dilations = [1,6,12,18]
        kernels   = [3,3,5,7]
        self.branches = nn.ModuleList()
        for d,k in zip(dilations,kernels):
            pad = (k//2)*d
            self.branches.append(SepConv(in_ch, out_ch, k, pad, d))
        self.merge = nn.Sequential(
            nn.Conv2d(len(dilations)*out_ch, out_ch, 1, bias=False),
            nn.GroupNorm(8, out_ch), nn.ReLU(),
            SEBlock(out_ch)
        )
    def forward(self,x):
        return self.merge(torch.cat([b(x) for b in self.branches], dim=1))

# -------------------------
# Residual DoubleConv + Strided Downsampling
# -------------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 5, padding=2, bias=False)
        self.norm1 = nn.GroupNorm(8, out_ch)
        self.act1  = nn.GELU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 5, padding=2, bias=False)
        self.norm2 = nn.GroupNorm(8, out_ch)
        self.act2  = nn.ReLU()
        self.res   = nn.Conv2d(in_ch, out_ch, 1, bias=False)
        self.se    = SEBlock(out_ch)
    def forward(self, x):
        residual = self.res(x)
        x = self.act1(self.norm1(self.conv1(x)))
        x = self.norm2(self.conv2(x))
        x = self.act2(x + residual)
        return self.se(x)

class PoolConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 5, stride=2, padding=2, bias=False),
            nn.GroupNorm(8, out_ch), nn.ReLU()
        )
    def forward(self, x): return self.net(x)

# -------------------------
# Bottleneck Transformer
# -------------------------
class BottleneckTransformer(nn.Module):
    def __init__(self, dim, heads=8, depth=6, mlp_dim=None):
        super().__init__()
        mlp_dim = mlp_dim or dim*4
        layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads,
                                           dim_feedforward=mlp_dim,
                                           activation='relu', norm_first=True)
        self.encoder = nn.TransformerEncoder(layer, depth)
        self.norm    = nn.LayerNorm(dim)
    def forward(self,x):
        bs,c,h,w = x.shape
        x = x.flatten(2).permute(2,0,1)
        x = self.encoder(x)
        x = x.permute(1,2,0).view(bs,c,h,w)
        return self.norm(x.permute(0,2,3,1)).permute(0,3,1,2)

# -------------------------
# Encoder/Decoder Blocks
# -------------------------
class EncoderBlock(nn.Module):
    def __init__(self,in_ch,out_ch,dropout=0.0,pool=True):
        super().__init__()
        self.conv = DoubleConv(in_ch, out_ch)
        self.down = PoolConv(out_ch, out_ch) if pool else None
        self.drop = nn.Dropout2d(dropout) if dropout>0 else None
    def forward(self,x):
        x = self.conv(x)
        if self.drop: x = self.drop(x)
        skip = x
        x = self.down(x) if self.down else x
        return x, skip

class DecoderBlock(nn.Module):
    def __init__(self,in_ch,out_ch,dropout=0.0):
        super().__init__()
        self.up   = nn.ConvTranspose2d(in_ch, out_ch, 2, stride=2)
        self.attn = AttentionGate(F_g=out_ch, F_l=out_ch, F_int=out_ch//2)
        self.conv = DoubleConv(out_ch*2, out_ch)
        self.drop = nn.Dropout2d(dropout) if dropout>0 else None
    def forward(self,x,skip):
        x = self.up(x)
        skip = self.attn(g=x, x=skip)
        x = self.conv(torch.cat([x,skip],dim=1))
        return self.drop(x) if self.drop else x

# -------------------------
# Full UNet with Attention and SE
# -------------------------
class UNetEnhanced(nn.Module):
    def __init__(self,in_ch=1,base=64,depth=4,dropout=0.0,ds=True):
        super().__init__()
        self.encs = nn.ModuleList()
        ch = in_ch
        for i in range(depth):
            out = base*(2**i)
            self.encs.append(EncoderBlock(ch, out, dropout, pool=(i<depth-1)))
            ch = out
        self.ms   = MultiScaleFusion(ch, ch)
        self.aspp = ASPP(ch, ch)
        self.trans= BottleneckTransformer(ch)
        self.decs = nn.ModuleList()
        ds_chs   = []
        for i in reversed(range(depth-1)):
            out = base*(2**i)
            self.decs.append(DecoderBlock(ch, out, dropout))
            ds_chs.append(out)
            ch = out
        self.final_seg = nn.Conv2d(ch, 1, kernel_size=1)
        self.ds = ds
        if ds:
            self.ds_heads = nn.ModuleList([
                nn.Conv2d(c, 1, kernel_size=1) for c in ds_chs
            ])
    def forward(self,x):
        skips = []
        for enc in self.encs:
            x, skip = enc(x)
            skips.append(skip)
        x = self.ms(x)
        x = self.aspp(x)
        x = self.trans(x)
        ds_out = []
        for idx, dec in enumerate(self.decs):
            x = dec(x, skips[-2-idx])
            if self.ds:
                ds_out.append(self.ds_heads[idx](x))
        seg_logits = self.final_seg(x)
        if self.ds:
            ds_out = [F.interpolate(o, size=seg_logits.shape[2:],
                         mode='bilinear', align_corners=False)
                      for o in ds_out]
            return seg_logits, ds_out
        return seg_logits, None

class TverskyF2Loss(nn.Module):
    def forward(self, preds, targets):
        TP = (preds*targets).sum()
        FN = ((1-preds)*targets).sum()
        FP = (preds*(1-targets)).sum()
        return (FN + 1e-6) / (2*TP + FN + 1e-6)

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.9, beta=0.1, gamma=2.0, eps=1e-6):
        super().__init__()
        self.alpha, self.beta, self.gamma, self.eps = alpha, beta, gamma, eps
    def forward(self, preds, targets):
        preds = preds.view(-1)
        targets = targets.view(-1)
        TP = (preds * targets).sum()
        FP = (preds * (1 - targets)).sum()
        FN = ((1 - preds) * targets).sum()
        tversky = (TP + self.eps) / (TP + self.alpha*FN + self.beta*FP + self.eps)
        return torch.pow((1 - tversky), self.gamma)

class ComboLoss(nn.Module):
    def __init__(self, pos_weight=500.0, bce_weight=0.2, dice_weight=0.4, f2_weight=0.4):
        super().__init__()
        self.bce  = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight))
        self.ft   = FocalTverskyLoss(alpha=0.9, beta=0.3, gamma=2.0)
        self.tf2 = TverskyF2Loss()
        self.bw, self.dw, self.fw= bce_weight, dice_weight, f2_weight
    def forward(self, logits, targets):
        bce_loss  = self.bce(logits, targets)
        dice_loss = self.ft(torch.sigmoid(logits), targets)
        f2_loss   = self.tf2(torch.sigmoid(logits), targets)
        return self.bw*bce_loss + self.dw*dice_loss + self.fw*f2_loss


class H5Dataset(Dataset):
    def __init__(self,path): self.path=path; self.f=None
    def __len__(self): return h5py.File(self.path,'r')['x'].shape[0]
    def __getitem__(self,idx):
        if self.f is None:
            self.f = h5py.File(self.path,'r', swmr=True)
        x = torch.from_numpy(self.f['x'][idx]).float()
        y = torch.from_numpy(self.f['y'][idx]).float()
        return x, y

In [8]:
if __name__ == '__main__':
    bs, max_batches, epochs = 5, 500, 200
    train_path = 'train.h5'
    val_path   = 'val.h5'

    # ─── datasets ───
    train_ds = H5Dataset(train_path)
    val_ds   = H5Dataset(val_path)

    # ─── train loader ───
    # pick exactly bs*max_batches random samples from train set
    small_idx = torch.randperm(len(train_ds))[:10]
    small_loader = DataLoader(train_ds, batch_size=5,
                          sampler=SubsetRandomSampler(small_idx))

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNetEnhanced(in_ch=1, base=32, depth=4, dropout=0.0, ds=True).to(device)
    crit  = ComboLoss(pos_weight=500, bce_weight=0.5, dice_weight=0.5, f2_weight=0.0)
    opt   = torch.optim.Adam(model.parameters(), lr=1e-4)
    sched = torch.optim.lr_scheduler.ConstantLR(opt, factor=1.0, total_iters=epochs)

    ds_weights = [1, 1, 1]

    for epoch in range(1, epochs+1):
        # ——— train ———
        model.train()
        total_loss = 0.0
        for batch_num, (imgs, msk) in enumerate(pbar:=tqdm(small_loader, desc=f"Epoch {epoch}")):
            #imgs[msk==1] = imgs[msk==1] * 3
            imgs = imgs/50
            imgs = imgs.to(device)
            msk  = msk.to(device)

            seg_logits, ds_out = model(imgs)
            loss = crit(seg_logits, msk)

            # deep supervision
            for aux, w in zip(ds_out, ds_weights):
                loss = loss + w * crit(aux, msk)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            pbar.set_postfix(avg_train_loss=total_loss/(batch_num+1), batch_loss=loss.item(), lr=sched.get_last_lr()[0])
        sched.step()

        # ——— validate ———
        model.eval()
        with torch.no_grad():
            tp=fp=fn=0
            loss = 0
            for imgs, msk in small_loader:
                #imgs[msk==1] = imgs[msk==1] * 3
                imgs = imgs/50
                imgs = imgs.to(device)
                msk  = msk.to(device)

                seg_logits, ds_out = model(imgs)
                loss += crit(seg_logits, msk)

                # deep supervision
                for aux, w in zip(ds_out, ds_weights):
                    loss = loss + w * crit(aux, msk)
                preds = (torch.sigmoid(seg_logits)>0.5).float().view(-1)
                t     = msk.view(-1)

                tp += (preds * t).sum().item()
                fp += (preds * (1-t)).sum().item()
                fn += ((1-preds)*t).sum().item()

            prec = tp/(tp+fp+1e-8)
            rec  = tp/(tp+fn+1e-8)
            f1   = 2*prec*rec/(prec+rec+1e-8)
            print(f"Val loss: {loss:.4f}  Val F1: {f1:.4f}  (P={prec:.4f}, R={rec:.4f})")


Epoch 1: 100%|██████████| 2/2 [00:00<00:00, 10.68it/s, avg_train_loss=5.53, batch_loss=6.02, lr=0.0001]


Val loss: 10.5824  Val F1: 0.0125  (P=0.0063, R=0.6698)


Epoch 2: 100%|██████████| 2/2 [00:00<00:00, 43.90it/s, avg_train_loss=5.26, batch_loss=4.91, lr=0.0001]


Val loss: 10.0373  Val F1: 0.0122  (P=0.0062, R=0.8862)


Epoch 3: 100%|██████████| 2/2 [00:00<00:00, 45.18it/s, avg_train_loss=4.99, batch_loss=5.26, lr=0.0001]


...

Val loss: 0.3935  Val F1: 0.6377  (P=0.4681, R=1.0000)


Epoch 199: 100%|██████████| 2/2 [00:00<00:00, 39.40it/s, avg_train_loss=0.174, batch_loss=0.182, lr=0.0001]


Val loss: 0.3385  Val F1: 0.6446  (P=0.4756, R=1.0000)


Epoch 200: 100%|██████████| 2/2 [00:00<00:00, 39.06it/s, avg_train_loss=0.163, batch_loss=0.161, lr=0.0001]


Val loss: 0.3146  Val F1: 0.6621  (P=0.4949, R=1.0000)


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    """
    A sequence of two 3x3 convolutions each followed by ReLU activation.
    """
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=True),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            #nn.GELU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=True),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
            #nn.GELU(),
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    """
    A basic U-Net architecture without attention, SE blocks, or deep supervision.
    """
    def __init__(self, in_channels=1, out_channels=1, features=None):
        super(UNet, self).__init__()
        if features is None:
            features = [64, 128, 256, 512]

        # Encoder path
        self.downs = nn.ModuleList()
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Bottleneck
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)

        # Decoder path
        self.ups = nn.ModuleList()
        rev_features = features[::-1]
        for feature in rev_features:
            self.ups.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=3, stride=2))
            self.ups.append(DoubleConv(feature * 2, feature))

        # Final 1x1 conv
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # Downsampling
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = F.max_pool2d(x, kernel_size=3, stride=2)

        # Bottleneck
        x = self.bottleneck(x)

        # Reverse skip connections for decoding
        skip_connections = skip_connections[::-1]

        # Upsampling
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)  # ConvTranspose2d
            skip = skip_connections[idx // 2]
            # In case the inexact sizes due to pooling
            if x.shape != skip.shape:
                x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=False)
            x = torch.cat((skip, x), dim=1)
            x = self.ups[idx + 1](x)  # DoubleConv

        return self.final_conv(x)


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
import h5py
from tqdm import tqdm
import random

class DiceLoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        num   = 2 * (probs * targets).sum(dim=(1,2,3))
        den   = probs.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3)) + self.eps
        dice  = num / den
        return 1 - dice.mean()

class BCEDiceLoss(nn.Module):
    def __init__(self, bce_weight=0.5):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()
        self.bw = bce_weight

    def forward(self, logits, targets):
        return self.bw * self.bce(logits, targets) + (1 - self.bw) * self.dice(logits, targets)

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, eps=1e-6):
        super().__init__()
        self.alpha, self.gamma, self.eps = alpha, gamma, eps

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits).clamp(self.eps, 1 - self.eps)
        pt = torch.where(targets==1, probs, 1 - probs)
        w  = torch.where(targets==1, self.alpha, 1 - self.alpha)
        loss = - w * (1 - pt)**self.gamma * pt.log()
        return loss.mean()

class TverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, eps=1e-6):
        super().__init__()
        self.alpha, self.beta, self.eps = alpha, beta, eps

    def forward(self, logits, targets):
        probs = torch.sigmoid(logits)
        TP = (probs * targets).sum(dim=(1,2,3))
        FP = (probs * (1-targets)).sum(dim=(1,2,3))
        FN = ((1-probs) * targets).sum(dim=(1,2,3))
        tversky = (TP + self.eps) / (TP + self.alpha*FN + self.beta*FP + self.eps)
        return 1 - tversky.mean()

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, gamma=1.5, eps=1e-6):
        super().__init__()
        self.tversky = TverskyLoss(alpha, beta, eps)
        self.gamma = gamma

    def forward(self, logits, targets):
        t = 1 - self.tversky(logits, targets)
        return t ** self.gamma

class TverskyF2Loss(nn.Module):
    def forward(self, preds, targets):
        TP = (preds*targets).sum()
        FN = ((1-preds)*targets).sum()
        FP = (preds*(1-targets)).sum()
        return (FN + 1e-6) / (2*TP + FN + 1e-6)

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.9, beta=0.1, gamma=2.0, eps=1e-6):
        super().__init__()
        self.alpha, self.beta, self.gamma, self.eps = alpha, beta, gamma, eps
    def forward(self, preds, targets):
        preds = preds.view(-1)
        targets = targets.view(-1)
        TP = (preds * targets).sum()
        FP = (preds * (1 - targets)).sum()
        FN = ((1 - preds) * targets).sum()
        tversky = (TP + self.eps) / (TP + self.alpha*FN + self.beta*FP + self.eps)
        return torch.pow((1 - tversky), self.gamma)

class ComboLoss(nn.Module):
    def __init__(self, pos_weight=500.0, bce_weight=0.2, dice_weight=0.4, f2_weight=0.4):
        super().__init__()
        self.bce  = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight))
        self.ft   = FocalTverskyLoss(alpha=0.9, beta=0.3, gamma=2.0)
        self.tf2 = TverskyF2Loss()
        self.bw, self.dw, self.fw= bce_weight, dice_weight, f2_weight
    def forward(self, logits, targets):
        bce_loss  = self.bce(logits, targets)
        dice_loss = self.ft(torch.sigmoid(logits), targets)
        f2_loss   = self.tf2(torch.sigmoid(logits), targets)
        return self.bw*bce_loss + self.dw*dice_loss + self.fw*f2_loss

class H5Dataset(Dataset):
    def __init__(self,path): self.path=path; self.f=None
    def __len__(self): return h5py.File(self.path,'r')['x'].shape[0]
    def __getitem__(self,idx):
        if self.f is None:
            self.f = h5py.File(self.path,'r', swmr=True)
        x = torch.from_numpy(self.f['x'][idx]).float()
        y = torch.from_numpy(self.f['y'][idx]).float()
        return x, y

In [ ]:
from torch.amp import GradScaler, autocast
if __name__ == '__main__':
    bs, max_batches, epochs = 5, 500, 200
    train_path = 'train.h5'
    val_path   = 'val.h5'

    # ─── datasets ───
    train_ds = H5Dataset(train_path)
    val_ds   = H5Dataset(val_path)

    # ─── train loader ───
    # pick exactly bs*max_batches random samples from train set
    # total small‐loader size and split
    num_samples = 50
    num_pos     = num_samples // 2
    num_neg     = num_samples - num_pos

    # collect all pos/neg indices in order
    pos_idx, neg_idx = [], []
    for i in range(len(train_ds)):
        _, mask = train_ds[i]
        if mask.sum() > 0:
            pos_idx.append(i)
        else:
            neg_idx.append(i)

    # take the first N from each
    selected_pos = pos_idx[:num_pos]
    selected_neg = neg_idx[:num_neg]

    # combine (optionally keep positives first, or interleave)
    balanced_idx = selected_pos + selected_neg

    small_loader = DataLoader(
        train_ds,
        batch_size=5,
        sampler=SubsetRandomSampler(balanced_idx),
        num_workers=8,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=4,
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNet(in_channels=1, out_channels=1).to(device)
    crit  = ComboLoss(pos_weight=100, bce_weight=0.5, dice_weight=0.5, f2_weight=0.0)
    #crit = DiceLoss()
    #crit = BCEDiceLoss(bce_weight=100)
    #crit = FocalLoss(alpha=0.25, gamma=2.0)
    #crit = FocalTverskyLoss(alpha=0.9, beta=0.1, gamma=1)

    opt   = torch.optim.Adam(model.parameters(), lr=1e-2)
    #sched = torch.optim.lr_scheduler.ConstantLR(opt, factor=1.0, total_iters=epochs)
    sched = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=5e-4, steps_per_epoch=len(small_loader), epochs=epochs)

    ds_weights = [1, 1, 1]
    scaler = GradScaler('cuda')

    for epoch in range(1, epochs+1):
        # ——— train ———
        model.train()
        total_loss = 0.0
        for batch_num, (imgs, msk) in enumerate(pbar:=tqdm(small_loader, desc=f"Epoch {epoch}")):
            #imgs[msk==1] = imgs[msk==1]
            imgs = imgs/50
            imgs = imgs.to(device)
            msk  = msk.to(device)
            opt.zero_grad()
            with autocast('cuda'):
                seg_logits = model(imgs)
                loss = crit(seg_logits, msk)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
            scaler.step(opt)
            scaler.update()
            #loss.backward()
            #opt.step()

            total_loss += loss.item()
            pbar.set_postfix(avg_train_loss=total_loss/(batch_num+1), batch_loss=loss.item(), lr=sched.get_last_lr()[0])
            sched.step()

        # ——— validate ———
        model.eval()
        with torch.no_grad():
            tp=fp=fn=0
            loss = 0
            for imgs, msk in small_loader:
                #imgs[msk==1] = imgs[msk==1]
                imgs = imgs/50
                imgs = imgs.to(device)
                msk  = msk.to(device)

                seg_logits = model(imgs)
                loss += crit(seg_logits, msk)
                preds = (torch.sigmoid(seg_logits)>0.5).float().view(-1)
                t     = msk.view(-1)

                tp += (preds * t).sum().item()
                fp += (preds * (1-t)).sum().item()
                fn += ((1-preds)*t).sum().item()

            prec = tp/(tp+fp+1e-8)
            rec  = tp/(tp+fn+1e-8)
            f1   = 2*prec*rec/(prec+rec+1e-8)
            print(f"Val loss: {loss:.4f}  Val F1: {f1:.4f}  (P={prec:.4f}, R={rec:.4f})")


Epoch 1: 100%|██████████| 10/10 [00:00<00:00, 31.33it/s, avg_train_loss=0.925, batch_loss=0.891, lr=2.03e-5]


Val loss: 9.1768  Val F1: 0.0039  (P=0.0020, R=0.9819)


Epoch 2: 100%|██████████| 10/10 [00:00<00:00, 49.69it/s, avg_train_loss=0.91, batch_loss=0.894, lr=2.12e-5]


Val loss: 9.1233  Val F1: 0.0051  (P=0.0025, R=0.7938)


Epoch 3: 100%|██████████| 10/10 [00:00<00:00, 51.98it/s, avg_train_loss=0.905, batch_loss=0.973, lr=2.28e-5]


...

Epoch 44: 100%|██████████| 10/10 [00:00<00:00, 52.16it/s, avg_train_loss=0.556, batch_loss=0.511, lr=0.00042]


Val loss: 5.4109  Val F1: 0.3423  (P=0.2106, R=0.9128)


Epoch 45: 100%|██████████| 10/10 [00:00<00:00, 48.13it/s, avg_train_loss=0.512, batch_loss=0.481, lr=0.000429]


Val loss: 5.3122  Val F1: 0.1909  (P=0.1063, R=0.9364)


Epoch 46:  50%|█████     | 5/10 [00:00<00:00, 46.28it/s, avg_train_loss=0.51, batch_loss=0.553, lr=0.000437] 

In [89]:
if __name__ == '__main__':
    bs, max_batches, epochs = 32, 500, 100
    train_path = 'train.h5'
    val_path   = 'val.h5'

    # ─── datasets ───
    train_ds = H5Dataset(train_path)
    val_ds   = H5Dataset(val_path)

    # ─── train loader ───
    # pick exactly bs*max_batches random samples from train set
    tr_idx = torch.randperm(len(train_ds))[: bs * max_batches]
    tr_loader = DataLoader(
        train_ds,
        batch_size=bs,
        sampler=SubsetRandomSampler(tr_idx),
        num_workers=8,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=4,
    )

    # ─── val loader ───
    # just iterate through val.h5 in order (or set shuffle=True if you like)
    val_loader = DataLoader(
        val_ds,
        batch_size=bs,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=4,
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNet(in_channels=1, out_channels=1).to(device)
    crit  = ComboLoss(pos_weight=100, bce_weight=0.5, dice_weight=0.5, f2_weight=0.0)
    #crit = DiceLoss()
    #crit = BCEDiceLoss(bce_weight=100)
    #crit = FocalLoss(alpha=0.25, gamma=2.0)
    #crit = FocalTverskyLoss(alpha=0.9, beta=0.1, gamma=1)

    opt   = torch.optim.Adam(model.parameters(), lr=1e-3)
    #sched = torch.optim.lr_scheduler.ConstantLR(opt, factor=1.0, total_iters=epochs)
    sched = torch.optim.lr_scheduler.OneCycleLR(
    opt, max_lr=1e-3,
    steps_per_epoch=len(tr_loader), epochs=epochs
)
    #sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=20, T_mult=3)

    ds_weights = [1, 1, 1]
    scaler = GradScaler('cuda')

    for epoch in range(1, epochs+1):
        # ——— train ———
        model.train()
        total_loss = 0.0
        for batch_num, (imgs, msk) in enumerate(pbar:=tqdm(tr_loader, desc=f"Epoch {epoch}")):
            #imgs[msk==1] = imgs[msk==1] * 3
            imgs = imgs/50
            imgs = imgs.to(device)
            msk  = msk.to(device)
            opt.zero_grad()
            with autocast('cuda'):
                seg_logits = model(imgs)
                loss = crit(seg_logits, msk)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0)
            scaler.step(opt)
            scaler.update()
            #loss.backward()
            #opt.step()

            total_loss += loss.item()
            pbar.set_postfix(avg_train_loss=total_loss/(batch_num+1), batch_loss=loss.item(), lr=sched.get_last_lr()[0])
            sched.step()

        # ——— validate ———
        model.eval()
        with torch.no_grad():
            tp=fp=fn=0
            loss = 0
            for imgs, msk in val_loader:
                #imgs[msk==1] = imgs[msk==1] * 3
                imgs = imgs/50
                imgs = imgs.to(device)
                msk  = msk.to(device)

                seg_logits = model(imgs)
                loss += crit(seg_logits, msk)
                preds = (torch.sigmoid(seg_logits)>0.5).float().view(-1)
                t     = msk.view(-1)

                tp += (preds * t).sum().item()
                fp += (preds * (1-t)).sum().item()
                fn += ((1-preds)*t).sum().item()

            prec = tp/(tp+fp+1e-8)
            rec  = tp/(tp+fn+1e-8)
            f1   = 2*prec*rec/(prec+rec+1e-8)
            print(f"Val loss: {loss:.4f}  Val F1: {f1:.4f}  (P={prec:.4f}, R={rec:.4f})")


Epoch 1:   0%|          | 0/469 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.64 GiB. GPU 0 has a total capacity of 15.48 GiB of which 1.59 GiB is free. Process 2857 has 258.00 MiB memory in use. Including non-PyTorch memory, this process has 12.85 GiB memory in use. Of the allocated memory 9.55 GiB is allocated by PyTorch, and 2.96 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)